In [1]:
from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage.cumulativebuildingdamage import CumulativeBuildingDamage

In [2]:
# Connect to IN-CORE service 
client = IncoreClient()
fragility_service = FragilityService(client)        # loading fragility mapping

Connection successful to IN-CORE services. pyIncore version detected: 0.9.0


### Building Damage due to EQ

In [3]:
bldg_dataset_id = "5df40388b9219c06cf8b0c80"        # defining building dataset (GIS point layer)

eq_mapping_id = "5d2789dbb9219c3c553c7977" 
eq_100_yr = "5dfa4058b9219c934b64d495"      # 100 yr EQ
hazard_type = "earthquake"
eq_dmg_result_name = 'bldg_eq_100yr_dmg_result'

In [4]:
eq_bldg_dmg = BuildingDamage(client)                   # initializing pyincore
eq_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id) # loading in the above
eq_mapping_set = MappingSet(fragility_service.get_mapping(eq_mapping_id))
eq_bldg_dmg.set_input_dataset("dfr3_mapping_set", eq_mapping_set)
   
eq_bldg_dmg.set_parameter("hazard_type", hazard_type)  # defining hazard type (e.g. earthquake vs. tsunami)
eq_bldg_dmg.set_parameter("hazard_id", eq_100_yr)      # loading above into pyincore
eq_bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
eq_bldg_dmg.set_parameter("num_cpu", 4)                # number of CPUs to use for parallel processing
eq_bldg_dmg.set_parameter("result_name", eq_dmg_result_name)  # specifying output name in pyincore

eq_bldg_dmg.run_analysis()                             # running the analysis with the above parameters

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [5]:
eq_bldg_dmg_output = eq_bldg_dmg.get_output_dataset('ds_result')

### Building Damage due to Tsunami

In [6]:
tsunami_100_yr = "5bc9e25ef7b08533c7e610dc"    # 100 yr Tsunami

ts_mapping_id = "5d279bb9b9219c3c553c7fba" 
hazard_type = "tsunami"
ts_dmg_result_name = 'bldg_tsu_100yr_dmg_result'

In [7]:
ts_bldg_dmg = BuildingDamage(client)                   # initializing pyincore
ts_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id) # loading in the above
ts_mapping_set = MappingSet(fragility_service.get_mapping(ts_mapping_id))
ts_bldg_dmg.set_input_dataset("dfr3_mapping_set", ts_mapping_set)
   
ts_bldg_dmg.set_parameter("hazard_type", hazard_type)  # defining hazard type (e.g. earthquake vs. tsunami)
ts_bldg_dmg.set_parameter("hazard_id", tsunami_100_yr)      # loading above into pyincore
ts_bldg_dmg.set_parameter("num_cpu", 4)                # number of CPUs to use for parallel processing
ts_bldg_dmg.set_parameter("result_name", ts_dmg_result_name)  # specifying output name in pyincore

ts_bldg_dmg.run_analysis()  

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [8]:
ts_bldg_dmg_output = ts_bldg_dmg.get_output_dataset('ds_result')

### Cumulative Building Damage Analysis for Seaside, OR

In [9]:
# Create cumulative Building Damage
cumulative_bldg_dmg = CumulativeBuildingDamage(client)

# Set input datasets
cumulative_bldg_dmg.set_input_dataset("eq_bldg_dmg", eq_bldg_dmg_output)
cumulative_bldg_dmg.set_input_dataset("tsunami_bldg_dmg", ts_bldg_dmg_output)

# Specify the result name
result_name = "cumulative_bldg_dmg_result"

# Set analysis parameters
cumulative_bldg_dmg.set_parameter("result_name", result_name)
cumulative_bldg_dmg.set_parameter("num_cpu", 4)

True

In [10]:
# Run Cumulative Building Damage Analysis
cumulative_bldg_dmg.run_analysis()

True

In [11]:
# Retrieve result dataset
result = cumulative_bldg_dmg.get_output_dataset("combined-result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()

,guid,LS_0,LS_1,LS_2,DS_0,DS_1,DS_2,DS_3,hazard
0,fba4c73f-9b1b-4942-b7fb-bc276a4e0a4e,0.080076,0.007494,0.000484,0.919924,0.072582,0.007010,0.000484,Earthquake+Tsunami
1,65d6e370-c05b-4b2f-bd17-d914d1d056c7,0.041585,0.002802,0.000086,0.958415,0.038783,0.002717,0.000086,Earthquake+Tsunami
2,dad76485-9694-409e-ac7f-c64c7bd5c32c,0.080076,0.007494,0.000484,0.919924,0.072582,0.007010,0.000484,Earthquake+Tsunami
3,6f57cfb6-d244-40d5-96f5-03aeddd5e30a,0.041585,0.002802,0.000086,0.958415,0.038783,0.002717,0.000086,Earthquake+Tsunami
4,929f1040-6224-409d-b4e3-58946daf3b91,0.080076,0.007494,0.000484,0.919924,0.072582,0.007010,0.000484,Earthquake+Tsunami
